In [50]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [28]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_il.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0"], axis =1) 
fips_check.head() #10912

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,9724667708,5/1/20,101,PPP,FERNANDINA BEACH HOTEL GROUP LLC,55 East Jackson Blvd,Chicago,IL,NaN,8/17/21,...,Hotels (except Casino Hotels) and Motels,999 Other,7211 Traveler Accommodation,7211.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465
1,1060027105,4/9/20,507,PPP,"MVP WORKFORCE, LLC",1751 Lake Cook Road Suite 600,DEERFIELD,IL,60015.0,7/17/21,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512
2,1255837210,4/15/20,507,PPP,"PLS GROUP, INC.","One South Wacker Drive, 36th Floor",CHICAGO,IL,60606.0,10/19/21,...,Other Activities Related to Credit Intermediat...,999 Other,5223 Activities Related to Credit Intermediation,5223.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672
3,1314447103,4/10/20,507,PPP,RELIABLE STAFFING SERVICES INC,870 Oak ST. 0.0,Lake in the Hills,IL,60156.0,NaN,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248
4,2083627203,4/15/20,507,PPP,"LAUNCH TECHNICAL WORKFORCE SOLUTIONS, LLC",700 Commerce Drive Suite 140,Oak Brook,IL,60523.0,10/20/21,...,Aircraft Manufacturing,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622


In [29]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [30]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan 

,LoanNumber,full_add,Lat,Long
9815,8342537301,"7000 S ADAMS ST STE 250, WILLOW BROOK, IL",NaN,NaN
33678,2821508310,"847, EVANSTON, IL",NaN,NaN


In [31]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #10904

,LoanNumber,full_add,Lat,Long
0,9724667708,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465
1,1060027105,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512
2,1255837210,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672
3,1314447103,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248
4,2083627203,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/IL_fips_scraped.csv") 
fips_check_scraped.head()

In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #40813

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #1239

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('17')] #IL
            fips_elements =[element for element in startswith if len(element)==15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

40813


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1239]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-11-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,9724667708,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,[170313204001021]
1,1060027105,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,[170318016011002]
2,1255837210,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,[170318391002007]
3,1314447103,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,[171118712051000]
4,2083627203,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,[170438446011053]
...,...,...,...,...,...
40810,9752217002,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,[170438400002003]
40811,9780317408,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,[170438458082006]
40812,9883677109,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,[170318046033000]
40813,9887958310,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,[170318036083005]


Remove brackets

In [12]:
#fips_fill['FIPS_z'] = geo_id
fips_loc = fips_fill
#coordinates with more than one return:
test = fips_loc[fips_loc['FIPS_z'].str.len() > 1]
test

,LoanNumber,full_add,Lat,Long,FIPS_z
46,6786427000,"1501 RIVER DR, MOLINE, IL",41.508526,-90.517157,"[179494934492020, 171610223001041]"
274,8853117001,"120 E Clark Street, FREEPORT, IL",42.301466,-89.620088,"[179494934492020, 171770008003002]"
286,6899737006,"300 Hamilton Blvd, PEORIA, IL",40.693498,-89.588734,"[179494934492020, 171430012002031]"
317,2412247100,"1200 W STATE ST, ROCKFORD, IL",42.275120,-89.108312,"[179494934492020, 172010026002007]"
337,7881847000,"766 Research Parkway, ROCKFORD, IL",42.218582,-89.085242,"[179494934492020, 172019800001003]"
...,...,...,...,...,...
40590,1642518302,"201 E 3rd St, Sterling, IL",41.788415,-89.692748,"[179494934492020, 171950011022079]"
40601,1977747100,"201 N 6th St, ROCKFORD, IL",42.268593,-89.081328,"[179494934492020, 172010011001000]"
40692,4755497204,"4370 7th Street, MOLINE, IL",41.467211,-90.526233,"[179494934492020, 171610219001063]"
40712,6000388301,"621 Main St, Hillsdale, IL",41.612486,-90.190552,"[179494934492020, 171610201011013]"


In [ ]:
test[['FIPS_w','FIPS_x', 'FIPS_y', 'FIPS_z']]  = test['FIPS_z'].apply(lambda x: pd.Series(str(x).split(",")))
test 

In [ ]:
pd.set_option('display.max_rows', 1600)
test

In [ ]:
test = test.drop(['FIPS_w','FIPS_z'], axis=1)
test["FIPS_z"] = test["FIPS_y"]
test['FIPS_z']= test['FIPS_z'].replace(']', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace(' ', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace("'", '', regex=True)
test

In [ ]:
test['FIPS_z']= test['FIPS_z'].fillna(test['FIPS_x'])
test['FIPS_z']= test['FIPS_z'].replace(']', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace(' ', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace("'", '', regex=True)
test

In [ ]:
cols = ['LoanNumber', 'FIPS_z']
test = test[cols]
test

In [32]:
fips_check_notna = fips_check_notna.merge(test, on="LoanNumber", how="left")
fips_check_notna

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9724667708,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,NaN
1,1060027105,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,NaN
2,1255837210,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,NaN
3,1314447103,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,NaN
4,2083627203,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,NaN
...,...,...,...,...,...
40808,9752217002,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,NaN
40809,9780317408,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,NaN
40810,9883677109,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,NaN
40811,9887958310,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,NaN


In [35]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9724667708,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,170313204001021
1,1060027105,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,170318016011002
2,1255837210,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,170318391002007
3,1314447103,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,171118712051000
4,2083627203,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,170438446011053
...,...,...,...,...,...
40810,9752217002,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,170438400002003
40811,9780317408,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,170438458082006
40812,9883677109,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,170318046033000
40813,9887958310,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,170318036083005


In [36]:
fips_fill= fips_fill[cols]
fips_fill

,LoanNumber,FIPS_z
0,9724667708,170313204001021
1,1060027105,170318016011002
2,1255837210,170318391002007
3,1314447103,171118712051000
4,2083627203,170438446011053
...,...,...
40810,9752217002,170438400002003
40811,9780317408,170438458082006
40812,9883677109,170318046033000
40813,9887958310,170318036083005


In [37]:
fips_check_notna = fips_check_notna.merge(fips_fill, on="LoanNumber", how="left")

In [40]:
fips_check_notna["FIPS_z"] = fips_check_notna["FIPS_z_x"].fillna(fips_check_notna["FIPS_z_y"])
fips_check_notna = fips_check_notna.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_check_notna

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9724667708,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,170313204001021
1,1060027105,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,170318016011002
2,1255837210,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,170318391002007
3,1314447103,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,171118712051000
4,2083627203,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,170438446011053
...,...,...,...,...,...
40808,9752217002,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,170438400002003
40809,9780317408,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,170438458082006
40810,9883677109,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,170318046033000
40811,9887958310,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,170318036083005


In [41]:
fips_check_notna.loc[fips_check_notna["FIPS_z"].isnull()] #check

,LoanNumber,full_add,Lat,Long,FIPS_z
1330,4357417000,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1978,6227707103,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
2002,4428188509,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2088,3984847201,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3153,2252087100,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3197,3313488304,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3559,6478417409,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3631,6829277004,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3813,6463967004,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN
3844,7639977104,"7505 S Us Highway 66, GARDNER, IL",36.145096,-96.002364,NaN


Parse for Merge

In [42]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_check_notna[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,9724667708,170313204001021
1,1060027105,170318016011002
2,1255837210,170318391002007
3,1314447103,171118712051000
4,2083627203,170438446011053
...,...,...
40808,9752217002,170438400002003
40809,9780317408,170438458082006
40810,9883677109,170318046033000
40811,9887958310,170318036083005


Read in base data

In [43]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_il.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/IL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #15796


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,11,11,9724667708,5/1/20,101,PPP,FERNANDINA BEACH HOTEL GROUP LLC,55 East Jackson Blvd,Chicago,IL,...,Hotels (except Casino Hotels) and Motels,999 Other,7211 Traveler Accommodation,7211.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465
1,309918,309918,1060027105,4/9/20,507,PPP,"MVP WORKFORCE, LLC",1751 Lake Cook Road Suite 600,DEERFIELD,IL,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512
2,309919,309919,1255837210,4/15/20,507,PPP,"PLS GROUP, INC.","One South Wacker Drive, 36th Floor",CHICAGO,IL,...,Other Activities Related to Credit Intermediat...,999 Other,5223 Activities Related to Credit Intermediation,5223.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672
3,309920,309920,1314447103,4/10/20,507,PPP,RELIABLE STAFFING SERVICES INC,870 Oak ST. 0.0,Lake in the Hills,IL,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248
4,309921,309921,2083627203,4/15/20,507,PPP,"LAUNCH TECHNICAL WORKFORCE SOLUTIONS, LLC",700 Commerce Drive Suite 140,Oak Brook,IL,...,Aircraft Manufacturing,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40810,350727,350727,9752217002,4/9/20,507,PPP,"D & S WIRE, INC.",2531 E. Devon Ave,ELK GROVE VILLAGE,IL,...,Steel Wire Drawing,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377
40811,350728,350728,9780317408,5/20/20,507,PPP,"PREFERRED METAL TECHNOLOGIES, INC.",9341 Adam Don Pkwy,WOODRIDGE,IL,...,"Plumbing, Heating, and Air-Conditioning Contra...",999 Other,2382 Building Equipment Contractors,2382.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",250 to 499,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819
40812,350729,350729,9883677109,4/15/20,507,PPP,"CORE PSYCHOTHERAPY CENTER, LTD.",1305 WILEY ROAD STE 125,SCHAUMBURG,IL,...,"Offices of Physicians, Mental Health Specialists",999 Other,6211 Offices of Physicians,6211.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553
40813,350730,350730,9887958310,1/31/21,507,PPS,THE ARLINGTON AUTOMOTIVE GROUP INC,2095 N Rand Rd,Palatine,IL,...,New Car Dealers,999 Other,4411 Automobile Dealers,4411.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347


Merge with MainFrame

In [44]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,11,11,9724667708,5/1/20,101,PPP,FERNANDINA BEACH HOTEL GROUP LLC,55 East Jackson Blvd,Chicago,IL,...,999 Other,7211 Traveler Accommodation,7211.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,170313204001021
1,309918,309918,1060027105,4/9/20,507,PPP,"MVP WORKFORCE, LLC",1751 Lake Cook Road Suite 600,DEERFIELD,IL,...,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,170318016011002
2,309919,309919,1255837210,4/15/20,507,PPP,"PLS GROUP, INC.","One South Wacker Drive, 36th Floor",CHICAGO,IL,...,999 Other,5223 Activities Related to Credit Intermediation,5223.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,170318391002007
3,309920,309920,1314447103,4/10/20,507,PPP,RELIABLE STAFFING SERVICES INC,870 Oak ST. 0.0,Lake in the Hills,IL,...,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,171118712051000
4,309921,309921,2083627203,4/15/20,507,PPP,"LAUNCH TECHNICAL WORKFORCE SOLUTIONS, LLC",700 Commerce Drive Suite 140,Oak Brook,IL,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,170438446011053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40810,350727,350727,9752217002,4/9/20,507,PPP,"D & S WIRE, INC.",2531 E. Devon Ave,ELK GROVE VILLAGE,IL,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,170438400002003
40811,350728,350728,9780317408,5/20/20,507,PPP,"PREFERRED METAL TECHNOLOGIES, INC.",9341 Adam Don Pkwy,WOODRIDGE,IL,...,999 Other,2382 Building Equipment Contractors,2382.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",250 to 499,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,170438458082006
40812,350729,350729,9883677109,4/15/20,507,PPP,"CORE PSYCHOTHERAPY CENTER, LTD.",1305 WILEY ROAD STE 125,SCHAUMBURG,IL,...,999 Other,6211 Offices of Physicians,6211.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,170318046033000
40813,350730,350730,9887958310,1/31/21,507,PPS,THE ARLINGTON AUTOMOTIVE GROUP INC,2095 N Rand Rd,Palatine,IL,...,999 Other,4411 Automobile Dealers,4411.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,170318036083005


If FIPS Encoded

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

Convert to File

In [45]:
geo_IL_fips = fips_merge

In [54]:
geo_IL_fips.to_csv("data/state_data/geo/geo_fips/IL_fips_scraped.csv") 

Review & Compare

In [53]:
geo_IL_fips.loc[geo_IL_fips["LoanNumber"] == 8477327904, "Lat"] = 37.37855
geo_IL_fips.loc[geo_IL_fips["LoanNumber"] == 8477327904, "Long"] = -89.161644
geo_IL_fips.loc[geo_IL_fips["LoanNumber"] == 8477327904, "FIPS_z"] = 171819505002071

In [51]:
geo_IL_fips = pd.read_csv("data/state_data/geo/geo_fips/IL_fips_scraped.csv") 
geo_IL_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,11,11,9724667708,5/1/20,101,PPP,FERNANDINA BEACH HOTEL GROUP LLC,55 East Jackson Blvd,Chicago,...,999 Other,7211 Traveler Accommodation,7211.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"55 East Jackson Blvd , Chicago, IL",41.878011,-87.625465,1.703132e+14
1,1,309918,309918,1060027105,4/9/20,507,PPP,"MVP WORKFORCE, LLC",1751 Lake Cook Road Suite 600,DEERFIELD,...,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14
2,2,309919,309919,1255837210,4/15/20,507,PPP,"PLS GROUP, INC.","One South Wacker Drive, 36th Floor",CHICAGO,...,999 Other,5223 Activities Related to Credit Intermediation,5223.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14
3,3,309920,309920,1314447103,4/10/20,507,PPP,RELIABLE STAFFING SERVICES INC,870 Oak ST. 0.0,Lake in the Hills,...,999 Other,5613 Employment Services,5613.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14
4,4,309921,309921,2083627203,4/15/20,507,PPP,"LAUNCH TECHNICAL WORKFORCE SOLUTIONS, LLC",700 Commerce Drive Suite 140,Oak Brook,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40810,40810,350727,350727,9752217002,4/9/20,507,PPP,"D & S WIRE, INC.",2531 E. Devon Ave,ELK GROVE VILLAGE,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,1.704384e+14
40811,40811,350728,350728,9780317408,5/20/20,507,PPP,"PREFERRED METAL TECHNOLOGIES, INC.",9341 Adam Don Pkwy,WOODRIDGE,...,999 Other,2382 Building Equipment Contractors,2382.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",250 to 499,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,1.704385e+14
40812,40812,350729,350729,9883677109,4/15/20,507,PPP,"CORE PSYCHOTHERAPY CENTER, LTD.",1305 WILEY ROAD STE 125,SCHAUMBURG,...,999 Other,6211 Offices of Physicians,6211.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,1.703180e+14
40813,40813,350730,350730,9887958310,1/31/21,507,PPS,THE ARLINGTON AUTOMOTIVE GROUP INC,2095 N Rand Rd,Palatine,...,999 Other,4411 Automobile Dealers,4411.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,1.703180e+14


In [52]:
#fill in missing Lat, Long, FIPS
geo_IL_fips.loc[geo_IL_fips["FIPS_z"].isnull()] #36

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
1330,1330,311247,311247,4357417000,4/3/20,507,PPP,"LEIBOLD IRRIGATION, INC.",18960 County Hwy 5 W,EAST DUBUQUE,...,999 Other,"2213 Water, Sewage and Other Systems",2213.0,ILLINOIS DISTRICT OFFICE,d $2-5 million,100 to 249,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1978,1978,311895,311895,6227707103,4/14/20,507,PPP,BUREAU SERVICE COMPANY,22069 US HIGHWAY 34,PRINCETON,...,999 Other,4249 Miscellaneous Nondurable Goods Merchant W...,4249.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
2002,2002,311919,311919,4428188509,2/25/21,507,PPS,BUREAU SERVICE COMPANY,22069 US Highway 34,Princeton,...,999 Other,4249 Miscellaneous Nondurable Goods Merchant W...,4249.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2088,2088,312005,312005,3984847201,4/27/20,507,PPP,"MINUTEMAN, INC.",14 N 845 US-20,Hampshire,...,333 Machinery,3333 Commercial and Service Industry Machinery...,3333.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3153,3153,313070,313070,2252087100,4/10/20,507,PPP,"HAMPSHIRE HOLDINGS, LLC",14 N 850 Route 20,HAMPSHIRE,...,999 Other,4442 Lawn and Garden Equipment and Supplies St...,4442.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,250 to 499,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3197,3197,313114,313114,3313488304,1/21/21,507,PPS,CAMOSY INCORPORATED,43451 N US Highway 41,Zion,...,999 Other,2362 Nonresidential Building Construction,2362.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,50 to 99,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3559,3559,313476,313476,6478417409,5/14/20,507,PPP,WAYNE-WHITE COUNTIES ELECTRIC COOPERATIVE,1501 W. MAIN ST PO Drawer E,FAIRFIELD,...,2211 Electric Utilities,"2211 Electric Power Generation, Transmission a...",2211.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,20 to 49,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3631,3631,313548,313548,6829277004,4/7/20,507,PPP,CAMOSY INCORPORATED,43451 US HIGHWAY 41,ZION,...,999 Other,2362 Nonresidential Building Construction,2362.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,50 to 99,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3813,3813,313730,313730,6463967004,4/7/20,507,PPP,CALCO,11600 Sterling Parkway,RICHMOND,...,321 Wood Products,3219 Other Wood Product Manufacturing,3219.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN
3844,3844,313761,313761,7639977104,4/14/20,507,PPP,"DIBBLE TRUCKING, INC.",7505 S Us Highway 66,GARDNER,...,999 Other,4841 General Freight Trucking,4841.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"7505 S Us Highway 66, GARDNER, IL",36.145096,-96.002364,NaN


<b>Value Re-verification - (if no pairing found in state aggregation)

In [19]:
#['17491', '17697', '17731', '17766', '17859', '17949', '17987']
geo_il_ver = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17949']
geo_il_ver2 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17941']
geo_il_ver3 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17697']
geo_il_ver4 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17731']
geo_il_ver5 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17766']
geo_il_ver6 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17859']
geo_il_ver7 = geo_IL_fips.loc[geo_IL_fips["FIPS_z"].astype(str).str[:5] == '17987']

frames = [geo_il_ver, geo_il_ver2, geo_il_ver3, geo_il_ver4, geo_il_ver5, geo_il_ver6, geo_il_ver7]
result = pd.concat(frames)
result

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
46,46,46,6786427000,4/7/20,507,PPP,"HOA HOLDINGS, LLC",1501 RIVER DR,MOLINE,IL,...,999 Other,7211 Traveler Accommodation,7211.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,500 or more,"1501 RIVER DR, MOLINE, IL",41.508526,-90.517157,1.794949e+14
274,274,274,8853117001,4/9/20,507,PPP,FURST-MCNESS COMPANY,120 E Clark Street,FREEPORT,IL,...,311 Food,3111 Animal Food Manufacturing,3111.0,ILLINOIS DISTRICT OFFICE,d $2-5 million,250 to 499,"120 E Clark Street, FREEPORT, IL",42.301466,-89.620088,1.794949e+14
286,286,286,6899737006,4/7/20,507,PPP,"HEYL, ROYSTER, VOELKER & ALLEN PC",300 Hamilton Blvd,PEORIA,IL,...,999 Other,5411 Legal Services,5411.0,ILLINOIS DISTRICT OFFICE,d $2-5 million,250 to 499,"300 Hamilton Blvd, PEORIA, IL",40.693498,-89.588734,1.794949e+14
317,317,317,2412247100,4/10/20,507,PPP,CRUSADERS CENTRAL CLINIC ASSOCIATION,1200 W STATE ST,ROCKFORD,IL,...,999 Other,6241 Individual and Family Services,6241.0,ILLINOIS DISTRICT OFFICE,d $2-5 million,250 to 499,"1200 W STATE ST, ROCKFORD, IL",42.275120,-89.108312,1.794949e+14
337,337,337,7881847000,4/8/20,507,PPP,"ROCKFORD TOOLCRAFT, INC.",766 Research Parkway,ROCKFORD,IL,...,332 Fabricated Metal Products,3321 Forging and Stamping,3321.0,ILLINOIS DISTRICT OFFICE,d $2-5 million,250 to 499,"766 Research Parkway, ROCKFORD, IL",42.218582,-89.085242,1.794949e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6343,6343,1526607105,4/10/20,507,PPP,"GRASSER'S PLUMBING & HEATING, INC.",404 W. MAIN ST,MC NABB,IL,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,ILLINOIS DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"404 W. MAIN ST, MC NABB, IL",41.176980,-89.209252,1.769799e+14
4969,4969,4969,8477327904,6/18/20,507,PPP,SOUTHERN ILLINOIS ELECTRIC COOPERATIVE,7420 US HIGHWAY 51 S,DONGOLA,IL,...,2211 Electric Utilities,"2211 Electric Power Generation, Transmission a...",2211.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,20 to 49,"7420 US HIGHWAY 51 S, DONGOLA, IL",35.828391,-89.451707,1.773142e+14
4878,4878,4878,5853367002,4/6/20,507,PPP,MCKAY AUTO PARTS INC,414 N OLD ROUTE 66,LITCHFIELD,IL,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,ILLINOIS DISTRICT OFFICE,c $1-2 million,100 to 249,"414 N OLD ROUTE 66, LITCHFIELD, IL",39.177663,-89.665934,1.776627e+14
40227,40227,40227,1847467210,4/15/20,507,PPP,"251 PUB, INC. DBA WOODS CREEK TAVERN",251 N RANDALL RD,LAKE IN THE HILLS,IL,...,999 Other,7224 Drinking Places (Alcoholic Beverages),7224.0,ILLINOIS DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"251 N RANDALL RD, LAKE IN THE HILLS, IL",42.178597,-88.334748,1.785969e+14
